# 1. Extraction of features from segmentation data

This first part of the analysis quantifies several features of the cells in 2D (analysis of each section of the stack individually): area of all segmented organelles per cell, number of mitochondria and nucleus, and shape of cell, mitochondria and nuclei. Shape is quantified as:

Aspect ratio: ratio between the longest and shortest sides of the bbox. The bbox is the minimal rectangle that contains the whole object in terms of its coordenates.
Roundness: ratio between the object´s area and its longest side (FH) of the bbox:
$$roundness = \frac{4*A}{\pi*F_{H}^{2}}$$

## Export segmentation
The script uses segmentation files exported from Vast Lite using the following conditions:

- Multi-tile stack
- Mip level: 0
- Export segmentation as ".png"
- 8bit/px
- Collapsed labels as in current view
- Export currently selected label (and select each segmen layer for each organelle manually).

Each of the segments for each organelle has to be stored in a different folder and the name of the individual files must contain ".vsseg_export_s". After "s", it detects the number of the section using the following, so that if the section has two digits zfill(2) and if it has three digits zfill(3):

- self.file = r'*export_s{}*'.format(str(sl_num).zfill(...))
For the tile size: recommended to use a divisible number to the total height and width of the section.

In case of changing the tile size: need to change the numbers on Cells.py (in patchsize inside of the function definition get_imageshape) and the number of tiles that will be created from that into self.imageshape (first two values).
For example: if for a image of 1024x1024 the tile size selected is 256x256, it would create four tiles, so:

- patchsize=(256,256)
- self.imageshape = (4,4,*patchsize)

In [1]:
from Cells import Cells
from MGFeatures import *
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import napari
from tifffile import imsave
import skimage.measure
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

C:\Users\Gast-User\.conda\envs\MG\lib\site-packages\numba\core\decorators.py:253: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
# TO-DO: include this definition in Cells.py or MGFeatures.py

def get_area(labels, labels_list): 
    areas = []
    for i in labels_list:
        area = np.count_nonzero(labels == i)
        areas.append(area)
    areas = np.array(areas)
    return areas

In [3]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell_final'
heterochromatin_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\heterochromatin_final'
gold_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\gold_final'
lysosomes_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\lysosome_final'
mitochondria_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\mitochondria_final'
nucleus_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\nucleus_final'
ER_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\er_final'
golgi_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\golgi_final'

##  tcell_properties function

The following functionautomatically extracts several features of the segments per label, being here a label one cell in each plane. The code is divided into different functions:

- #### A. Label cells

This part of the code creates the labels for the cell segment. It runs over the i, which is the section number. The functions indicated: 'get_imageshape()', 'img_from_tiles()', 'label_cells()' are described in Cells.py. These functions reconstruct the whole image from the tiles created for the segmentation export and detects the connected elements to label the individual cells. 'cell_labels' is an array that can be visualized with napari.

- #### B: Label organelles

This part of the code creates individual labels for the organelles, but they are dependent of the cell label via the mask_it() function. This function returns an array np.int8, in which the information is the number of the label of the cell.
For example, if a cell with label number 1 has two mitochondria, both of these mitochondria will be labeled 1.

- #### C: Connected labels

The skimage.measure.label() with connectivity=2 creates labelling of the organelles in a different way, so that two mitochondria present in the same cell contain a different label. This is not necessary for the cell labels, but it changes the data to an array of np.int64 in all cases.

- #### D: Dataframe with cell, mitochondria and nucleus coordinates

For the cell, mitochondria and nucleus labels, a pandas data frame is created that contains per label the coordinates of the bbox, stored as the minimum and maximum values of x and y axis.

- #### E. Assign mitochondria and nucleus to cell coordinates

The loop assigns the label of the cell to which each organelle belongs and stores it as 'id_cell', so that the organelles that belong to the same cell can later be grouped and extracted the mean values.

- #### F. Calculate properties per cell

Using the coordinates of each label we calculate the aspect ratio and roundness. For the aspect ratio, the largest and smallest side of the bbox are calculated as the maximum and minimum difference, respectively, between the row and column values. 

- #### G. Add properties to data frame

All properties are added to an empty data frame. In the case of properties related to mitochondria and nucleus, the mean properties of the organelles is calculated per cell, by grouping them by their cell id.

In [4]:
def tcell_properties(i):
    #propertiesall  = pd.DataFrame()
    # A. Label cells
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) #TODO: this step is redundant! Fix in cells
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    labels_list = np.unique(cell_labels)[1:]
    
    # B. label organelles
    heterochromatin_labels = mask_it(cell_labels, cell.img_from_tiles(heterochromatin_folder))
    lysosomes_labels = mask_it(cell_labels, cell.img_from_tiles(lysosomes_folder))
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    gold_labels = mask_it(cell_labels, cell.img_from_tiles(gold_folder))
    golgi_labels = mask_it(cell_labels, cell.img_from_tiles(golgi_folder))
    ER = cell.img_from_tiles(ER_folder)
    
    # C. Connected labels
    cell_label = skimage.measure.label(cell_labels, connectivity=2)
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    
    #D. Dataframe with cell, mitochondria and nucleus coordinates
    cell_coord = []
    coord_cell = pd.DataFrame()
    region = regionprops(cell_label)
    for r in region:
        min_row, min_col, max_row, max_col = r.bbox
        cell_coord.append((min_row, max_row, min_col, max_col))
    coord_cell = pd.DataFrame(cell_coord, columns = ["min_row","max_row", "min_col", "max_col"])
   
    mito_coord = []
    coord_mito = pd.DataFrame()
    region = regionprops(mito_label)
    for r in region:
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col))
    coord_mito = pd.DataFrame(mito_coord, columns = ["min_row", "min_col", "max_row", "max_col"])
    
    nuc_coord = []
    coord_nuc = pd.DataFrame()
    region = regionprops(nuc_label)
    for r in region:
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col))
    coord_nuc = pd.DataFrame(nuc_coord, columns = ["min_row", "min_col", "max_row", "max_col"])
    
    #E. Assign mitochondria and nucleus to cell coordinates
    coord_mito['id_cell'] = 0
    for x in range(len(coord_cell)):
        for y in range(len(coord_mito)):
            if (coord_mito["min_row"][y] >= coord_cell["min_row"][x]  and coord_mito["max_row"][y] <= coord_cell["max_row"][x]) and (coord_mito["min_col"][y] >= coord_cell["min_col"][x]and coord_mito["max_col"][y] <= coord_cell["max_col"][x]):
                coord_mito['id_cell'][y] = x
    
    coord_nuc['id_cell'] = 0
    for y in range(len(coord_nuc)):
        for x in range(len(coord_cell)):
            if (coord_nuc["min_row"][y] >= coord_cell["min_row"][x]  and coord_nuc["max_row"][y] <= coord_cell["max_row"][x]) and (coord_nuc["min_col"][y] >= coord_cell["min_col"][x]and coord_nuc["max_col"][y] <= coord_cell["max_col"][x]):
                coord_nuc['id_cell'][y] = x
    
    #F. Calculate properties per cell
    #F1. Mitochondria
    coord_mito['mito_large'] = 0
    coord_mito['mito_small'] = 0
    for y in range(len(coord_mito)):
        D = max(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
        coord_mito['mito_large'][y] = D
        d = min(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
        coord_mito['mito_small'][y] = d
    coord_mito['mito_aspect'] = coord_mito['mito_small']/coord_mito['mito_large']
    mito_list = np.unique(mito_label)[1:]
    mito_area = get_area(mito_label, mito_list)
    coord_mito['mito_area'] = mito_area
    coord_mito['mito_roundness'] = (4*coord_mito['mito_area'])/(np.pi*(coord_mito['mito_large']**2))
    
    #F2. Nucleus
    coord_nuc['nuc_large'] = 0
    coord_nuc['nuc_small'] = 0
    for y in range(len(coord_nuc)):
        M = max(coord_nuc['max_row'][y] - coord_nuc['min_row'][y], coord_nuc['max_col'][y] - coord_nuc['min_col'][y])
        coord_nuc['nuc_large'][y] = M
        m = min(coord_nuc['max_row'][y] - coord_nuc['min_row'][y], coord_nuc['max_col'][y] - coord_nuc['min_col'][y])
        coord_nuc['nuc_small'][y] = m
    coord_nuc['nuc_aspect'] = coord_nuc['nuc_small']/coord_nuc['nuc_large']
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area = get_area(nuc_label, nuc_list)
    coord_nuc['nuc_area'] = nuc_area
    coord_nuc['nuc_roundness'] = (4*coord_nuc['nuc_area'])/(np.pi*(coord_nuc['nuc_large']**2))
    
    #F3. Cell
    coord_cell['cell_large'] = 0
    coord_cell['cell_small'] = 0
    for y in range(len(coord_cell)):
        D = max(coord_cell['max_row'][y] - coord_cell['min_row'][y], coord_cell['max_col'][y] - coord_cell['min_col'][y])
        coord_cell['cell_large'][y] = D
        d = min(coord_cell['max_row'][y] - coord_cell['min_row'][y], coord_cell['max_col'][y] - coord_cell['min_col'][y])
        coord_cell['cell_small'][y] = d
    coord_cell['cell_aspect'] = coord_cell['cell_small']/coord_cell['cell_large']
    
    #G. Add to properties data frame
    properties = pd.DataFrame()
    cell_list = np.unique(cell_labels)[1:]
    properties['cell_area'] = get_area(cell_labels, cell_list)
    properties['cell_aspect'] = coord_cell['cell_aspect']
    properties['cell_roundness'] = (4*properties['cell_area'])/(np.pi*(coord_cell['cell_large']**2))
    properties['number_mito'] = coord_mito.groupby(['id_cell']).size()
    properties['mean_mito_aspect'] = coord_mito.groupby(['id_cell'])['mito_aspect'].mean()
    properties['std_mito_aspect'] = coord_mito.groupby(['id_cell'])['mito_aspect'].std()
    properties['mean_mito_area'] = coord_mito.groupby(['id_cell'])['mito_area'].mean()
    properties['std_mito_area'] = coord_mito.groupby(['id_cell'])['mito_area'].std()
    properties['mean_mito_roundness'] = coord_mito.groupby(['id_cell'])['mito_roundness'].mean()
    properties['std_mito_roundness'] = coord_mito.groupby(['id_cell'])['mito_roundness'].std()
    properties['sum_mito_area'] = coord_mito.groupby(['id_cell'])['mito_area'].sum()
    properties['number_nuc'] = coord_nuc.groupby(['id_cell']).size()
    properties['mean_nuc_aspect'] = coord_nuc.groupby(['id_cell'])['nuc_aspect'].mean()
    properties['std_nuc_aspect'] = coord_nuc.groupby(['id_cell'])['nuc_aspect'].std()
    properties['mean_nuc_area'] = coord_nuc.groupby(['id_cell'])['nuc_area'].mean()
    properties['std_nuc_area'] = coord_nuc.groupby(['id_cell'])['nuc_area'].std()
    properties['mean_nuc_roundness'] = coord_nuc.groupby(['id_cell'])['nuc_roundness'].mean()
    properties['std_nuc_roundness'] = coord_nuc.groupby(['id_cell'])['nuc_roundness'].std()
    properties['std_nuc_area'] = coord_nuc.groupby(['id_cell'])['nuc_area'].std()
    properties['lysosomal_area'] = get_area(lysosomes_labels, labels_list)
    properties['heterochromatin_area'] = get_area(heterochromatin_labels, labels_list)
    properties['gold_area'] = get_area(gold_labels, labels_list)
    properties['nucleus_area'] = coord_nuc['nuc_area']
    properties['golgi_area'] = get_area(golgi_labels, labels_list)
    properties['cytoplasm_area'] = properties['cell_area'] - properties['nucleus_area']
    properties['ER_lengths'] = ER_length(ER, cell_labels)
    properties['sl_num'] = np.array([i]*len(labels_list))
    properties['x1'] = coord_cell['min_row']
    properties['x2'] = coord_cell['max_row']
    properties['y1'] = coord_cell['min_col']
    properties['y2'] = coord_cell['max_col']
    properties= properties.fillna(0)
    #propertiesall = propertiesall.append(properties)
    return properties

## Running the tcell_properties function in parallel
In order to make the process faster, the for loop to run each section using this function is achieved by running them in parallel. The nslices that can run in parallel in each case will depend on the number of cores of the pc and the total number in the range in the station´s RAM.

In [ ]:
start_time = time.time()
nslices = 6
properties0_5 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(5))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum0_5 = pd.DataFrame()
for i in range(5):
    propertiessum0_5= propertiessum0_5.append(properties0_5[i]) 
propertiessum0_5.to_excel("tcell_properties_0_5_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties5_10 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(5,10))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum5_10 = pd.DataFrame()
for i in range(5):
    propertiessum5_10= propertiessum5_10.append(properties5_10[i]) 
propertiessum5_10.to_excel("tcell_properties_5_10_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties10_15 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(10,15))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum10_15 = pd.DataFrame()
for i in range(5):
    propertiessum10_15= propertiessum10_15.append(properties10_15[i]) 
propertiessum10_15.to_excel("tcell_properties_10_15_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties15_20 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(15,20))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum15_20 = pd.DataFrame()
for i in range(5):
    propertiessum15_20= propertiessum15_20.append(properties15_20[i]) 
propertiessum15_20.to_excel("tcell_properties_15_20_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties20_25 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(20,25))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum20_25 = pd.DataFrame()
for i in range(5):
    propertiessum20_25= propertiessum20_25.append(properties20_25[i]) 
propertiessum20_25.to_excel("tcell_properties_20_25_v4.xlsx")

## Start from here

In [5]:
start_time = time.time()
nslices = 5
properties25_30 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(25,30))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 16.3min remaining: 24.4min


Elapsed time:  2112.881951570511


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 35.2min finished


In [6]:
propertiessum25_30 = pd.DataFrame()
for i in range(5):
    propertiessum25_30= propertiessum25_30.append(properties25_30[i]) 
propertiessum25_30.to_excel("tcell_properties_25_30_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2626863135.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum25_30= propertiessum25_30.append(properties25_30[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2626863135.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum25_30= propertiessum25_30.append(properties25_30[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2626863135.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum25_30= propertiessum25_30.append(properties25_30[i])


In [7]:
start_time = time.time()
nslices = 5
properties30_35 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(30,35))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 14.7min remaining: 22.1min


Elapsed time:  954.7967352867126


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 15.9min finished


In [8]:
propertiessum30_35 = pd.DataFrame()
for i in range(5):
    propertiessum30_35= propertiessum30_35.append(properties30_35[i]) 
propertiessum30_35.to_excel("tcell_properties_30_35_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\512546378.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum30_35= propertiessum30_35.append(properties30_35[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\512546378.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum30_35= propertiessum30_35.append(properties30_35[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\512546378.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum30_35= propertiessum30_35.append(properties30_35[i])


In [9]:
start_time = time.time()
nslices = 5
properties35_40 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(35,40))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 13.6min remaining: 20.4min


Elapsed time:  931.2061541080475


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 15.5min finished


In [10]:
propertiessum35_40 = pd.DataFrame()
for i in range(5):
    propertiessum35_40= propertiessum35_40.append(properties35_40[i]) 
propertiessum35_40.to_excel("tcell_properties_35_40_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3265785951.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum35_40= propertiessum35_40.append(properties35_40[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3265785951.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum35_40= propertiessum35_40.append(properties35_40[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3265785951.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum35_40= propertiessum35_40.append(properties35_40[i])


In [11]:
start_time = time.time()
nslices = 5
properties40_45 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(40,45))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 10.5min remaining: 15.8min


Elapsed time:  792.4029803276062


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 13.2min finished


In [12]:
propertiessum40_45 = pd.DataFrame()
for i in range(5):
    propertiessum40_45= propertiessum40_45.append(properties40_45[i]) 
propertiessum40_45.to_excel("tcell_properties_40_45_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3284945451.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum40_45= propertiessum40_45.append(properties40_45[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3284945451.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum40_45= propertiessum40_45.append(properties40_45[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3284945451.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum40_45= propertiessum40_45.append(properties40_45[i])


In [13]:
start_time = time.time()
nslices = 5
properties45_50 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(45,50))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 10.4min remaining: 15.7min


Elapsed time:  831.9728043079376


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 13.9min finished


In [14]:
propertiessum45_50 = pd.DataFrame()
for i in range(5):
    propertiessum45_50= propertiessum45_50.append(properties45_50[i]) 
propertiessum45_50.to_excel("tcell_properties_45_50_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3156594406.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum45_50= propertiessum45_50.append(properties45_50[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3156594406.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum45_50= propertiessum45_50.append(properties45_50[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\3156594406.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum45_50= propertiessum45_50.append(properties45_50[i])


In [15]:
start_time = time.time()
nslices = 5
properties50_55 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(50,55))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  8.0min remaining: 11.9min


Elapsed time:  514.4088485240936


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  8.6min finished


In [16]:
propertiessum50_55 = pd.DataFrame()
for i in range(5):
    propertiessum50_55= propertiessum50_55.append(properties50_55[i]) 
propertiessum50_55.to_excel("tcell_properties_50_55_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2998702726.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum50_55= propertiessum50_55.append(properties50_55[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2998702726.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum50_55= propertiessum50_55.append(properties50_55[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2998702726.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum50_55= propertiessum50_55.append(properties50_55[i])


In [17]:
start_time = time.time()
nslices = 5
properties55_60 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(55,60))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  7.6min remaining: 11.4min


Elapsed time:  485.8726234436035


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  8.1min finished


In [18]:
propertiessum55_60 = pd.DataFrame()
for i in range(5):
    propertiessum55_60= propertiessum55_60.append(properties55_60[i]) 
propertiessum55_60.to_excel("tcell_properties_55_60_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\1160059991.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum55_60= propertiessum55_60.append(properties55_60[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\1160059991.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum55_60= propertiessum55_60.append(properties55_60[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\1160059991.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum55_60= propertiessum55_60.append(properties55_60[i])


In [19]:
start_time = time.time()
nslices = 5
properties60_65 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(60,65))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  8.9min remaining: 13.3min


Elapsed time:  1005.5485219955444


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 16.8min finished


In [20]:
propertiessum60_65 = pd.DataFrame()
for i in range(5):
    propertiessum60_65= propertiessum60_65.append(properties60_65[i]) 
propertiessum60_65.to_excel("tcell_properties_60_65_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2365184860.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum60_65= propertiessum60_65.append(properties60_65[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2365184860.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum60_65= propertiessum60_65.append(properties60_65[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2365184860.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum60_65= propertiessum60_65.append(properties60_65[i])


In [21]:
start_time = time.time()
nslices = 5
properties65_70 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(65,70))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  8.2min remaining: 12.3min


Elapsed time:  512.316899061203


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  8.5min finished


In [22]:
propertiessum65_70 = pd.DataFrame()
for i in range(5):
    propertiessum65_70= propertiessum65_70.append(properties65_70[i]) 
propertiessum65_70.to_excel("tcell_properties_65_70_v4.xlsx")

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2606102559.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum65_70= propertiessum65_70.append(properties65_70[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2606102559.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum65_70= propertiessum65_70.append(properties65_70[i])
C:\Users\Gast-User\AppData\Local\Temp\ipykernel_3292\2606102559.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  propertiessum65_70= propertiessum65_70.append(properties65_70[i])


In [ ]:
start_time = time.time()
nslices = 5
properties70_75 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(70,75))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


In [ ]:
propertiessum70_75 = pd.DataFrame()
for i in range(5):
    propertiessum70_75= propertiessum70_75.append(properties70_75[i]) 
propertiessum70_75.to_excel("tcell_properties_70_75_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties75_80 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(75,80))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum75_80 = pd.DataFrame()
for i in range(5):
    propertiessum75_80= propertiessum75_80.append(properties75_80[i]) 
propertiessum75_80.to_excel("tcell_properties_75_80_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties80_85 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(80,85))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum80_85 = pd.DataFrame()
for i in range(5):
    propertiessum80_85= propertiessum80_85.append(properties80_85[i]) 
propertiessum80_85.to_excel("tcell_properties_80_85_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties85_90 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(85,90))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum85_90 = pd.DataFrame()
for i in range(5):
    propertiessum85_90= propertiessum85_90.append(properties85_90[i]) 
propertiessum85_90.to_excel("tcell_properties_85_90_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties90_95 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(90,95))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum90_95 = pd.DataFrame()
for i in range(5):
    propertiessum90_95= propertiessum90_95.append(properties90_95[i]) 
propertiessum90_95.to_excel("tcell_properties_90_95_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties95_100 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(95,100))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum95_100 = pd.DataFrame()
for i in range(5):
    propertiessum95_100= propertiessum95_100.append(properties95_100[i]) 
propertiessum95_100.to_excel("tcell_properties_95_100_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties100_105 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(100,105))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum100_105 = pd.DataFrame()
for i in range(5):
    propertiessum100_105= propertiessum100_105.append(properties100_105[i]) 
propertiessum100_105.to_excel("tcell_properties_100_105_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties105_110 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(105,110))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum105_110= pd.DataFrame()
for i in range(5):
    propertiessum105_110= propertiessum105_110.append(properties105_110[i]) 
propertiessum105_110.to_excel("tcell_properties_105_110_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties110_115 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(110,115))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum110_115= pd.DataFrame()
for i in range(5):
    propertiessum110_115= propertiessum110_115.append(properties110_115[i]) 
propertiessum110_115.to_excel("tcell_properties_110_115_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties115_120 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(115,120))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum115_120= pd.DataFrame()
for i in range(5):
    propertiessum115_120= propertiessum115_120.append(properties115_120[i]) 
propertiessum115_120.to_excel("tcell_properties_115_120_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties120_125 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(120,125))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum120_125= pd.DataFrame()
for i in range(5):
    propertiessum120_125= propertiessum120_125.append(properties120_125[i]) 
propertiessum120_125.to_excel("tcell_properties_120_125_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties125_130 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(125,130))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum125_130= pd.DataFrame()
for i in range(5):
    propertiessum125_130= propertiessum125_130.append(properties125_130[i]) 
propertiessum125_130.to_excel("tcell_properties_125_130_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties130_135 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(130,135))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum130_135= pd.DataFrame()
for i in range(5):
    propertiessum130_135= propertiessum130_135.append(properties130_135[i]) 
propertiessum130_135.to_excel("tcell_properties_130_135_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties135_140 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(135,140))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum135_140= pd.DataFrame()
for i in range(5):
    propertiessum135_140= propertiessum135_140.append(properties135_140[i]) 
propertiessum135_140.to_excel("tcell_properties_135_140_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties140_145 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(140,145))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum140_145= pd.DataFrame()
for i in range(5):
    propertiessum140_145= propertiessum140_145.append(properties140_145[i]) 
propertiessum140_145.to_excel("tcell_properties_140_145_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties145_150 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(145,150))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum145_150= pd.DataFrame()
for i in range(5):
    propertiessum145_150= propertiessum145_150.append(properties145_150[i]) 
propertiessum145_150.to_excel("tcell_properties_145_150_v4.xlsx")

In [ ]:
start_time = time.time()
nslices = 5
properties150_155 = Parallel(n_jobs=nslices, verbose=True)(delayed(tcell_properties)(i) for i in range(150,155))
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
propertiessum150_155= pd.DataFrame()
for i in range(5):
    propertiessum150_155= propertiessum150_155.append(properties150_155[i]) 
propertiessum150_155.to_excel("tcell_properties_150_155_v4.xlsx")